# Libraries 

In [4]:
import pandas as pd
import spacy
import nltk
import re
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split






In [2]:
nlp_urdu = spacy.blank("ur")
nltk.download('wordnet')
nltk.download('omw')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mahma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\mahma\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

# Load Data 

In [3]:
def load_urdu_sentiment_data(filename):
    tweets = []
    labels = []
    
    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip().split('\t')
            if len(line) == 2:
                tweets.append(line[0])
                labels.append(line[1])

    return tweets, labels

In [4]:
def read_conllu_file(file_path):
    words_roots = []
    with open(file_path, 'r', encoding='utf-8') as conllufile:
        lines = conllufile.readlines()
        for line in lines:
            if line.strip() and not line.startswith("#"): 
                parts = line.split('\t')
                if len(parts) >= 3:
                    word, root = parts[1], parts[2]
                    words_roots.append((word, root))
    return words_roots


In [5]:

file_path = 'sample.conllu'  
words_roots = read_conllu_file(file_path)
print(len(words_roots))
for word, root in words_roots:
    print("Word:", word, "\tRoot:", root)


3178
Word: صدر 	Root: صدر
Word: تلگودیشم 	Root: تلگودیشم
Word: مسٹر 	Root: مسٹر
Word: این 	Root: این
Word: چندرا 	Root: چندرا
Word: بابو 	Root: بابو
Word: نائیڈو 	Root: نائیڈو
Word: نے 	Root: نے
Word: کڑپہ 	Root: کڑپہ
Word: کے 	Root: کا
Word: ضمنی 	Root: ضمنی
Word: انتخابات 	Root: انتخاب
Word: مےں 	Root: مےں
Word: اضافی 	Root: اضافی
Word: فورسیس 	Root: فورسی
Word: کی 	Root: کا
Word: تعیناتی 	Root: تعیناتی
Word: کا 	Root: کا
Word: مطالبہ 	Root: مطالبہ
Word: کرتے 	Root: کر
Word: ہوئے 	Root: ہو
Word: چیف 	Root: چیف
Word: الیکشن 	Root: الیکشن
Word: کمشنر 	Root: کمشنر
Word: کو 	Root: کو
Word: مکتوب 	Root: مکتوب
Word: روانہ 	Root: روانہ
Word: کیا 	Root: کر
Word: ۔ 	Root: ۔
Word: انہوں 	Root: وہ
Word: نے 	Root: نے
Word: کڑپہ 	Root: کڑپہ
Word: کے 	Root: کا
Word: پارلیمانی 	Root: پارلیمانی
Word: اور 	Root: اور
Word: پولی 	Root: پولی
Word: ویندلہ 	Root: ویندلہ
Word: کے 	Root: کا
Word: اسمبلی 	Root: اسمبلی
Word: انتخابات 	Root: انتخاب
Word: کو 	Root: کو
Word: صاف 	Root: صاف
Word: و 	Root: و
Word:

# -------------------------------Pre Processing-------------------------

# Remove Punctuation

In [6]:
filename = 'urdu-sentiment-corpus-v1.tsv'
tweets, labels = load_urdu_sentiment_data(filename)
preprocessed_tweets = []
for tweet in tweets:
    preprocessed_tweet = re.sub(r'[؛۔٫٪+=@#!؟،۔)(}{]', '', tweet)
    preprocessed_tweets.append(preprocessed_tweet)

data = {'Tweet': preprocessed_tweets, 'Label': labels}
data = pd.DataFrame(data)
data

Tweet  Label
0                                                Tweet  Class
1    میں نے ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ ...      P
2    چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...      N
3                              ٹویٹر کا خیال کیسے آیا       O
4    سرچ انجن گوگل کے نائب صدر نے فضا میں  130,000 ...      P
..                                                 ...    ...
995      اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے       P
996  چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...      P
997  واہ جناب واہ اچھی رہی جناب خود کو فرشتہ سمجوں ...      P
998  اسلام آباد :پی اے ٹی کا دھرنا ختم صفائی کے کام...      P
999  دنیا نے کس کا راہ وفا میں دیا ہے ساتھتم بھی چل...      P

[1000 rows x 2 columns]

# Remove Numbers

In [7]:
urdu_digits = ['۶', '۴', '۵', '۸', '۲', '۰', '۷', '۹', '۳', '۱']

english_digits=['1','2','3','4','5','6','7','8','9','0']

def remove_numbers(text):
    pattern = '[' + ''.join(urdu_digits) + ''.join(english_digits) + ']'
    text = re.sub(pattern, '', text)
    return text

In [8]:
data['Tweet'] = data['Tweet'].apply(lambda x: remove_numbers(x))
data

Tweet  Label
0                                                Tweet  Class
1    میں نے ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ ...      P
2    چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...      N
3                              ٹویٹر کا خیال کیسے آیا       O
4    سرچ انجن گوگل کے نائب صدر نے فضا میں  , فٹ کی ...      P
..                                                 ...    ...
995      اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے       P
996  چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...      P
997  واہ جناب واہ اچھی رہی جناب خود کو فرشتہ سمجوں ...      P
998  اسلام آباد :پی اے ٹی کا دھرنا ختم صفائی کے کام...      P
999  دنیا نے کس کا راہ وفا میں دیا ہے ساتھتم بھی چل...      P

[1000 rows x 2 columns]

## A ) Identify Best Tokenizer

In [9]:
def compare_tokenizers(text):
    nltk_tokens = nltk.word_tokenize(text)
    nltk_vocab_size = len(set(nltk_tokens))
    nlp = spacy.blank("ur")
    spacy_doc = nlp(text)
    spacy_tokens = [token.text for token in spacy_doc]
    spacy_vocab_size = len(set(spacy_tokens))
    if nltk_vocab_size < spacy_vocab_size:
        best_tokenizer = "NLTK"
    elif nltk_vocab_size > spacy_vocab_size:
        best_tokenizer = "spaCy"
    
    nltk_errors = set([token for token in nltk_tokens if not token in spacy_tokens])
    spacy_errors = set([token for token in spacy_tokens if not token in nltk_tokens])

    print("NLTK Tokenization:", nltk_tokens)
    print("spaCy Tokenization:", spacy_tokens)
    print("\nNLTK Vocabulary Size:", nltk_vocab_size)
    print("spaCy Vocabulary Size:", spacy_vocab_size)
    print("\nNLTK Errors:", nltk_errors)
    print("spaCy Errors:", spacy_errors)
 

    return best_tokenizer
text = "میں نے ایٹم بم بنایا ہے۔ او بھائی ایٹم بمب کوٹ لکھپت والی اتفاق فیکٹری میں نہیں بنتا۔"
best_tokenizer = compare_tokenizers(text)
print("Best Tokenizer:", best_tokenizer)


NLTK Tokenization: ['میں', 'نے', 'ایٹم', 'بم', 'بنایا', 'ہے۔', 'او', 'بھائی', 'ایٹم', 'بمب', 'کوٹ', 'لکھپت', 'والی', 'اتفاق', 'فیکٹری', 'میں', 'نہیں', 'بنتا۔']
spaCy Tokenization: ['میں', 'نے', 'ایٹم', 'بم', 'بنایا', 'ہے', '۔', 'او', 'بھائی', 'ایٹم', 'بمب', 'کوٹ', 'لکھپت', 'والی', 'اتفاق', 'فیکٹری', 'میں', 'نہیں', 'بنتا', '۔']

NLTK Vocabulary Size: 16
spaCy Vocabulary Size: 17

NLTK Errors: {'ہے۔', 'بنتا۔'}
spaCy Errors: {'۔', 'ہے', 'بنتا'}
Best Tokenizer: NLTK


# B) Split Data

In [10]:
def tokenize_with_spacy(data):
    nlp = spacy.blank("ur")
    tokenized_data = []
    for tweet, label in zip(data['Tweet'], data['Label']):
        doc = nlp(tweet)
        tokenized_tweet = [(token.text, label) for token in doc]
        tokenized_data.extend(tokenized_tweet)
    return tokenized_data


In [11]:
tokenized_data = tokenize_with_spacy(data)
print("Total NUmber of Words ",len(tokenized_data))
tokenized_data

Total NUmber of Words  17639


[('Tweet', 'Class'),
 ('میں', 'P'),
 ('نے', 'P'),
 ('ایٹم', 'P'),
 ('بم', 'P'),
 ('بنایا', 'P'),
 ('ھے', 'P'),
 ('او', 'P'),
 ('بھائی', 'P'),
 ('ایٹم', 'P'),
 ('بمب', 'P'),
 ('کوٹ', 'P'),
 ('لکھپت', 'P'),
 ('والی', 'P'),
 ('اتفاق', 'P'),
 ('فیکٹری', 'P'),
 ('میں', 'P'),
 ('نہیں', 'P'),
 ('بنتاایٹم', 'P'),
 ('بم', 'P'),
 ('کہوٹہ', 'P'),
 ('کی', 'P'),
 ('ایٹمی', 'P'),
 ('چندے', 'N'),
 ('سے', 'N'),
 ('انقلاب', 'N'),
 ('اور', 'N'),
 ('عمران', 'N'),
 ('خان', 'N'),
 ('وزیر', 'N'),
 ('اعظم', 'N'),
 ('نہیں', 'N'),
 ('بن', 'N'),
 ('سکتے', 'N'),
 ('ٹویٹر', 'O'),
 ('کا', 'O'),
 ('خیال', 'O'),
 ('کیسے', 'O'),
 ('آیا', 'O'),
 ('سرچ', 'P'),
 ('انجن', 'P'),
 ('گوگل', 'P'),
 ('کے', 'P'),
 ('نائب', 'P'),
 ('صدر', 'P'),
 ('نے', 'P'),
 ('فضا', 'P'),
 ('میں', 'P'),
 (' ', 'P'),
 (',', 'P'),
 ('فٹ', 'P'),
 ('کی', 'P'),
 ('بلندی', 'P'),
 ('پر', 'P'),
 ('چھلانگ', 'P'),
 ('لگا', 'P'),
 ('کر', 'P'),
 ('عالمی', 'P'),
 ('ریکارڈ', 'P'),
 ('قائم', 'P'),
 ('کرلیا', 'P'),
 ('چھلانگ', 'P'),
 ('کی', 'P'),
 ('ابھی', 'P

 # C) Remove Stop Words 

In [12]:
def remove_stopwords_spacy_urdu(data):
    words = [word for word, _ in data]
    labels = [label for _, label in data]
    doc = nlp_urdu(" ".join(words))
    filtered_words = [token.text for token in doc if not token.is_stop and token.is_alpha]
    filtered_labels = [label for i, label in enumerate(labels) if i < len(doc) and not doc[i].is_stop and doc[i].is_alpha]
    merged_data = [(word, label) for word, label in zip(filtered_words, filtered_labels)]
    
    return merged_data


In [13]:
filtered_words = remove_stopwords_spacy_urdu(tokenized_data)
print("Total Words After removing Stop Words ",len(filtered_words))
filtered_words

Total Words After removing Stop Words  13592


[('Tweet', 'Class'),
 ('میں', 'P'),
 ('نے', 'P'),
 ('ایٹم', 'P'),
 ('بم', 'P'),
 ('بنایا', 'P'),
 ('ھے', 'P'),
 ('او', 'P'),
 ('بھائی', 'P'),
 ('ایٹم', 'P'),
 ('بمب', 'P'),
 ('کوٹ', 'P'),
 ('لکھپت', 'P'),
 ('والی', 'P'),
 ('اتفاق', 'P'),
 ('فیکٹری', 'P'),
 ('میں', 'P'),
 ('نہیں', 'P'),
 ('بنتاایٹم', 'P'),
 ('بم', 'P'),
 ('کہوٹہ', 'P'),
 ('ایٹمی', 'P'),
 ('چندے', 'N'),
 ('سے', 'N'),
 ('انقلاب', 'N'),
 ('عمران', 'N'),
 ('خان', 'N'),
 ('وزیر', 'N'),
 ('اعظم', 'N'),
 ('نہیں', 'N'),
 ('بن', 'N'),
 ('سکتے', 'N'),
 ('ٹویٹر', 'O'),
 ('کا', 'O'),
 ('خیال', 'O'),
 ('کیسے', 'O'),
 ('آیا', 'O'),
 ('سرچ', 'P'),
 ('انجن', 'P'),
 ('گوگل', 'P'),
 ('نائب', 'P'),
 ('صدر', 'P'),
 ('نے', 'P'),
 ('فضا', 'P'),
 ('میں', 'P'),
 ('فٹ', 'P'),
 ('بلندی', 'P'),
 ('چھلانگ', 'P'),
 ('لگا', 'P'),
 ('عالمی', 'P'),
 ('ریکارڈ', 'P'),
 ('قائم', 'P'),
 ('کرلیا', 'P'),
 ('چھلانگ', 'P'),
 ('ابھی', 'P'),
 ('اسکی', 'P'),
 ('لہریں', 'P'),
 ('کبھی', 'P'),
 ('کبھی', 'P'),
 ('آ', 'P'),
 ('جاتی', 'P'),
 ('یار', 'P'),
 ('گندی', 'N

# D  ) Lematization

In [14]:
def urdu_lemmatize_with_roots(words, words_roots):
    headers = words[0]  
    words_data = words[1:]  
    lemmatized_words = []
    for word_tuple in words_data:
        word = word_tuple[0]  
        found_root = False
        for w, root in words_roots:
            if word == w:
                lemmatized_words.append((root, word_tuple[1]))  
                found_root = True
                break
        if not found_root:
            lemmatized_words.append(word_tuple)  
    merged = [('Tweet', 'Class')] + lemmatized_words
    return merged


In [15]:

lemmatized_words = urdu_lemmatize_with_roots(filtered_words, words_roots)
print("Original Words:", filtered_words)
print("Lemmatized Words:", lemmatized_words)
lemmatized_df = pd.DataFrame(lemmatized_words, columns=['Tweet', 'Class'])



Original Words: [('Tweet', 'Class'), ('میں', 'P'), ('نے', 'P'), ('ایٹم', 'P'), ('بم', 'P'), ('بنایا', 'P'), ('ھے', 'P'), ('او', 'P'), ('بھائی', 'P'), ('ایٹم', 'P'), ('بمب', 'P'), ('کوٹ', 'P'), ('لکھپت', 'P'), ('والی', 'P'), ('اتفاق', 'P'), ('فیکٹری', 'P'), ('میں', 'P'), ('نہیں', 'P'), ('بنتاایٹم', 'P'), ('بم', 'P'), ('کہوٹہ', 'P'), ('ایٹمی', 'P'), ('چندے', 'N'), ('سے', 'N'), ('انقلاب', 'N'), ('عمران', 'N'), ('خان', 'N'), ('وزیر', 'N'), ('اعظم', 'N'), ('نہیں', 'N'), ('بن', 'N'), ('سکتے', 'N'), ('ٹویٹر', 'O'), ('کا', 'O'), ('خیال', 'O'), ('کیسے', 'O'), ('آیا', 'O'), ('سرچ', 'P'), ('انجن', 'P'), ('گوگل', 'P'), ('نائب', 'P'), ('صدر', 'P'), ('نے', 'P'), ('فضا', 'P'), ('میں', 'P'), ('فٹ', 'P'), ('بلندی', 'P'), ('چھلانگ', 'P'), ('لگا', 'P'), ('عالمی', 'P'), ('ریکارڈ', 'P'), ('قائم', 'P'), ('کرلیا', 'P'), ('چھلانگ', 'P'), ('ابھی', 'P'), ('اسکی', 'P'), ('لہریں', 'P'), ('کبھی', 'P'), ('کبھی', 'P'), ('آ', 'P'), ('جاتی', 'P'), ('یار', 'P'), ('گندی', 'N'), ('زبان', 'N'), ('گٹر', 'N'), ('جیسے', 'N')

In [16]:
lemmatized_df = lemmatized_df.iloc[1:]
lemmatized_df

Tweet Class
1         میں     P
2          نے     P
3        ایٹم     P
4          بم     P
5         بنا     P
...       ...   ...
13587  ساتھتم     P
13588     بھی     P
13589     یوں     P
13590      جب     P
13591     چلی     P

[13591 rows x 2 columns]

# -------------Sentiment analysis using deep learning--------------------

# RNN

In [17]:
lemmatized_df = lemmatized_df.dropna(subset=['Class'])
binary_labels = {'P': 1, 'N': 0, 'O': 0}
lemmatized_df['Class'] = lemmatized_df['Class'].map(binary_labels)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lemmatized_df['Tweet'])
X = tokenizer.texts_to_sequences(lemmatized_df['Tweet'])
max_sequence_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_sequence_length)
y = lemmatized_df['Class'].values


In [ ]:

def create_rnn_model(num_layers, dropout_rate, input_dim):
    model = Sequential()
    for _ in range(num_layers):
        model.add(SimpleRNN(64, return_sequences=True, input_shape=(input_dim, 1)))  
        model.add(Dropout(dropout_rate))
    model.add(SimpleRNN(64)) 
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax')) 
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
def train_and_evaluate_rnn_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, input_dim):
    model = create_rnn_model(num_layers, dropout_rate, input_dim)
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)  
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
num_layers_values = [2, 3]
dropout_rate_values = [0.3, 0.7]
results = {}
for num_layers in num_layers_values:
    for dropout_rate in dropout_rate_values:
        accuracy, precision, recall, f1 = train_and_evaluate_rnn_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, X_train.shape[1])
        results[(num_layers, dropout_rate)] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}


In [37]:
print("Results for RNN models with different hyperparameters:")
print("------------------------------------------------------")
print("Num Layers\tDropout Rate\tAccuracy\tPrecision\tRecall\t\tF1-score")
for (num_layers, dropout_rate), metrics in results.items():
    print(f"{num_layers}\t\t{dropout_rate}\t\t{metrics['Accuracy']:.4f}\t\t{metrics['Precision']:.4f}\t\t{metrics['Recall']:.4f}\t\t{metrics['F1-score']:.4f}")


Results for RNN models with different hyperparameters:
------------------------------------------------------
Num Layers	Dropout Rate	Accuracy	Precision	Recall		F1-score
2		0.3		0.5061		0.3047		0.5061		0.3513
2		0.7		0.4912		0.3031		0.4912		0.3278
3		0.3		0.5113		0.3103		0.5113		0.3668
3		0.7		0.5065		0.2569		0.5065		0.3408


# GRU

In [ ]:
def create_gru_model(num_layers, dropout_rate, input_dim):
    model = Sequential()
    for _ in range(num_layers):
        model.add(GRU(64, return_sequences=True, input_shape=(input_dim, 1)))  
        model.add(Dropout(dropout_rate))
    model.add(GRU(64))  
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax')) 
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
def train_and_evaluate_gru_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, input_dim):
    model = create_gru_model(num_layers, dropout_rate, input_dim)
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)  
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
num_layers_values = [2, 3]
dropout_rate_values = [0.3, 0.7]

results = {}

for num_layers in num_layers_values:
    for dropout_rate in dropout_rate_values:
        accuracy, precision, recall, f1 = train_and_evaluate_gru_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, X_train.shape[1])
        results[(num_layers, dropout_rate)] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}


In [21]:
print("Results for GRU models with different hyperparameters:")
print("------------------------------------------------------")
print("Num Layers\tDropout Rate\tAccuracy\tPrecision\tRecall\t\tF1-score")
for (num_layers, dropout_rate), metrics in results.items():
    print(f"{num_layers}\t\t{dropout_rate}\t\t{metrics['Accuracy']:.4f}\t\t{metrics['Precision']:.4f}\t\t{metrics['Recall']:.4f}\t\t{metrics['F1-score']:.4f}")


Results for GRU models with different hyperparameters:
------------------------------------------------------
Num Layers	Dropout Rate	Accuracy	Precision	Recall		F1-score
2		0.3		0.5274		0.2781		0.5274		0.3642
2		0.7		0.5274		0.2781		0.5274		0.3642
3		0.3		0.5274		0.2781		0.5274		0.3642
3		0.7		0.5274		0.2781		0.5274		0.3642


# LSTM

In [24]:
def create_lstm_model(num_layers, dropout_rate, input_dim):
    model = Sequential()
    for _ in range(num_layers):
        model.add(LSTM(64, return_sequences=True, input_shape=(input_dim, 1)))  
        model.add(Dropout(dropout_rate))
    model.add(LSTM(64))  
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
def train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, input_dim):
    model = create_lstm_model(num_layers, dropout_rate, input_dim)
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1) 
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
num_layers_values = [2, 3]
dropout_rate_values = [0.3, 0.7]
results = {}
for num_layers in num_layers_values:
    for dropout_rate in dropout_rate_values:
        accuracy, precision, recall, f1 = train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, X_train.shape[1])
        results[(num_layers, dropout_rate)] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

# Print results
print("Results for LSTM models with different hyperparameters:")
print("------------------------------------------------------")
print("Num Layers\tDropout Rate\tAccuracy\tPrecision\tRecall\t\tF1-score")
for (num_layers, dropout_rate), metrics in results.items():
    print(f"{num_layers}\t\t{dropout_rate}\t\t{metrics['Accuracy']:.4f}\t\t{metrics['Precision']:.4f}\t\t{metrics['Recall']:.4f}\t\t{metrics['F1-score']:.4f}")


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step
Results for LSTM models with different hyperparameters:
------------------------------------------------------
Num Layers	Dropout Rate	Accuracy	Precision	Recall		F1-score
2		0.3		0.4726		0.2234		0.4726		0.3034
2		0.7		0.5274		0.2781		0.5274		0.3642
3		0.3		0.5274		0.2781		0.5274		0.3642
3		0.7		0.5274		0.2781		0.5274		0.3642


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# BILSTM

In [25]:
def create_bilstm_model(num_layers, dropout_rate, input_dim):
    model = Sequential()
    for _ in range(num_layers):
        model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(input_dim, 1)))  
        model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(64)))  
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))  
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
def train_and_evaluate_bilstm_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, input_dim):
    model = create_bilstm_model(num_layers, dropout_rate, input_dim)
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=0)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1) 
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
num_layers_values = [2, 3]
dropout_rate_values = [0.3, 0.7]
results = {}
for num_layers in num_layers_values:
    for dropout_rate in dropout_rate_values:
        accuracy, precision, recall, f1 = train_and_evaluate_bilstm_model(X_train, y_train, X_test, y_test, num_layers, dropout_rate, X_train.shape[1])
        results[(num_layers, dropout_rate)] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}
print("Results for BiLSTM models with different hyperparameters:")
print("------------------------------------------------------")
print("Num Layers\tDropout Rate\tAccuracy\tPrecision\tRecall\t\tF1-score")
for (num_layers, dropout_rate), metrics in results.items():
    print(f"{num_layers}\t\t{dropout_rate}\t\t{metrics['Accuracy']:.4f}\t\t{metrics['Precision']:.4f}\t\t{metrics['Recall']:.4f}\t\t{metrics['F1-score']:.4f}")


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step
Results for BiLSTM models with different hyperparameters:
------------------------------------------------------
Num Layers	Dropout Rate	Accuracy	Precision	Recall		F1-score
2		0.3		0.4726		0.2234		0.4726		0.3034
2		0.7		0.5274		0.2781		0.5274		0.3642
3		0.3		0.5274		0.2781		0.5274		0.3642
3		0.7		0.5274		0.2781		0.5274		0.3642


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
